In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import StratifiedKFold

In [2]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from sklearn.model_selection import cross_val_score

In [4]:
train = pd.read_csv('../data/train.csv')

In [5]:
test = pd.read_csv('../data/test.csv')

In [6]:
train.shape

(1000000, 16)

In [7]:
train.head()

,customer,category,suspicious,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit
0,90000000,0,0,734958.58,7.0,0.142857,79.0,134,0,24.63,124.0,3,6.0,0.128703,215959.85,832.364854
1,90000001,1,0,1009400.35,2.0,0.500000,0.0,33,0,78.49,79.0,3,1.0,0.635584,53.97,34.752298
2,90000002,0,0,11051.68,3.0,0.666667,77.0,191,0,35.74,72.0,2,1.0,0.138152,1632.91,17.323031
3,90000003,0,0,31193.29,81.0,0.308642,77.0,50,0,2.05,95.0,2,2.0,0.333904,3203.10,50.945281
4,90000004,0,0,205489.77,1.0,1.000000,39.0,172,0,147.53,148.0,4,1.0,0.341578,5626.65,0.000000


In [8]:
Counter(train['suspicious'].values)

Counter({0: 990913, 1: 9087})

In [9]:
test.head()

,customer,category,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit
0,91000000,0,1235859.85,1.0,0.00,64.0,33,0,106.42,107.0,2,1.0,0.046108,1235859.85,12549.438865
1,91000001,1,53087.97,3.0,0.00,0.0,172,0,69.28,139.0,1,1.0,0.414826,53087.97,116.106050
2,91000002,0,22003.25,4.0,0.25,41.0,33,0,30.94,62.0,2,1.0,0.030105,1066.98,106.434228
3,91000003,0,72947.07,4.0,0.50,16.0,94,0,24.19,97.0,2,1.0,0.025249,16701.34,106.455505
4,91000004,0,9964.06,1.0,0.00,46.0,50,0,47.84,48.0,2,1.0,0.083395,9964.06,118.541699


In [10]:
part1 = train[train['suspicious'] ==1]
part1.shape

(9087, 16)

In [11]:
part2 = train[train['suspicious'] ==0].head(part1.shape[0])
part2.shape

(9087, 16)

In [12]:
part1.describe()

,customer,category,suspicious,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit
count,9.087000e+03,9087.000000,9087.0,9.087000e+03,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9.087000e+03,9.087000e+03
mean,9.049759e+07,0.132497,1.0,1.929921e+07,45.579069,0.371553,40.611423,90.662705,0.006383,43.592635,113.594145,1.808848,9.840541,0.285898,2.556440e+06,7.219705e+04
std,2.910666e+05,0.410150,0.0,4.024280e+08,87.923922,0.308070,18.828287,61.522094,0.079641,52.730146,76.851286,0.905809,26.918985,0.168321,7.610242e+07,1.838689e+06
min,9.000014e+07,0.000000,1.0,1.500000e-01,1.000000,0.000000,0.000000,18.000000,0.000000,0.220000,2.000000,1.000000,1.000000,0.000784,0.000000e+00,0.000000e+00
25%,9.024361e+07,0.000000,1.0,2.123016e+05,1.000000,0.000000,31.000000,33.000000,0.000000,3.485000,74.000000,1.000000,1.000000,0.155065,2.988150e+03,6.483963e+01
50%,9.049522e+07,0.000000,1.0,5.510467e+05,3.000000,0.355405,42.000000,50.000000,0.000000,27.380000,95.000000,2.000000,1.000000,0.261564,2.407594e+04,7.028325e+02
75%,9.075139e+07,0.000000,1.0,1.416317e+06,51.000000,0.500000,53.000000,172.000000,0.000000,62.815000,128.500000,2.000000,5.000000,0.390301,1.156037e+05,3.697764e+03
max,9.099999e+07,2.000000,1.0,3.313022e+10,870.000000,1.000000,131.000000,191.000000,1.000000,496.630000,994.000000,8.000000,670.000000,0.997642,4.309531e+09,1.601670e+08


In [13]:
part2.describe()

,customer,category,suspicious,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit
count,9.087000e+03,9087.000000,9087.0,9.087000e+03,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9087.000000,9.087000e+03,9.087000e+03
mean,9.000458e+07,0.249037,0.0,1.228093e+07,22.483768,0.320948,36.479586,87.573897,0.010234,46.602068,115.340046,2.006713,5.062177,0.250174,1.578653e+06,3.928894e+04
std,2.648270e+03,0.536314,0.0,1.383877e+08,44.103920,0.310984,24.372413,60.216412,0.100652,52.728013,78.522153,0.998491,12.897025,0.166573,5.564425e+07,7.042401e+05
min,9.000000e+07,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,18.000000,0.000000,0.350000,1.000000,1.000000,0.000000,0.000952,0.000000e+00,0.000000e+00
25%,9.000229e+07,0.000000,0.0,4.174485e+04,1.000000,0.000000,18.000000,33.000000,0.000000,7.110000,72.000000,1.000000,1.000000,0.124421,1.446000e+02,1.083694e+01
50%,9.000458e+07,0.000000,0.0,1.483924e+05,3.000000,0.297872,39.000000,50.000000,0.000000,29.850000,96.000000,2.000000,1.000000,0.215314,3.317740e+03,1.899042e+02
75%,9.000688e+07,0.000000,0.0,4.535044e+05,23.000000,0.500000,56.000000,172.000000,0.000000,65.550000,134.000000,3.000000,3.000000,0.338287,2.386657e+04,1.187544e+03
max,9.000917e+07,2.000000,0.0,7.302745e+09,487.000000,1.000000,110.000000,191.000000,1.000000,462.050000,848.000000,7.000000,346.000000,0.975229,4.636720e+09,3.880817e+07


In [14]:
training_stratified_min = part1.append(part2)
training_stratified_min.shape

(18174, 16)

In [15]:
training_stratified_min.head()

,customer,category,suspicious,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit
137,90000137,0,1,505063.29,234.0,0.320513,48.0,33,0,0.81,90.0,2,102.0,0.231895,6206.19,2214.228651
302,90000302,0,1,476155.65,7.0,0.714286,70.0,50,0,17.83,107.0,1,2.0,0.240542,51301.67,214.683563
346,90000346,0,1,1065190.66,1.0,0.000000,54.0,32,0,44.46,89.0,1,1.0,0.692185,0.00,3383.820679
388,90000388,0,1,1735813.48,82.0,0.329268,38.0,134,0,2.05,95.0,2,1.0,0.267895,542527.80,12704.901736
396,90000396,0,1,98422.93,2.0,0.000000,67.0,32,0,48.78,147.0,5,1.0,0.494655,98422.93,72.603265


In [16]:
Counter(training_stratified_min['suspicious'].values)

Counter({0: 9087, 1: 9087})

In [17]:
# X = training_stratified_min[['turnover','atm_withdrawal', 'transaction_count', 'io_ratio']].values
X = training_stratified_min.drop(['suspicious'],axis=1)

In [18]:
X = X.values

In [55]:
X_full = train.drop(['suspicious'],axis=1).values

In [59]:
y_full = train['suspicious'].values

In [60]:
y_full.shape

(1000000,)

In [56]:
X_full.shape

(1000000, 15)

In [19]:
y = training_stratified_min['suspicious'].values

In [20]:
Counter(y)

Counter({0: 9087, 1: 9087})

In [21]:
X.shape

(18174, 15)

In [22]:
y.shape

(18174,)

In [23]:
clf = LogisticRegression(random_state=0)

In [24]:
X_ = test.values

In [25]:
X_.shape

(1000000, 15)

In [26]:
clf2 = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial')

In [51]:
knn = KNeighborsClassifier(n_neighbors=25)

In [41]:
svc_linear = SVC(kernel="linear", C=0.025)
svc = SVC(gamma=2, C=1)

In [42]:
X.shape

(18174, 15)

In [43]:
y.shape

(18174,)

In [57]:
cross_val_score(knn,X,y,cv=5)

array([0.67051705, 0.97057206, 0.97798569, 0.97165658, 0.96477711])

In [61]:
knn.fit(X_full,y_full)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=25, p=2,
           weights='uniform')

In [62]:
knn_res = knn.predict_proba(X_)
knn_res

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.92, 0.08],
       [1.  , 0.  ],
       [1.  , 0.  ]])

In [69]:
knn_res[0:500]

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.96, 0.04],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.96, 0.04],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.92, 0.08],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.96, 0.04],
       [1.  , 0.  ],
       [0.96, 0.04],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  ,

In [63]:
knn_res.shape

(1000000, 2)

In [67]:
knn_res[:,1].max()

0.24

In [70]:
Counter(knn_res[:,1])

Counter({0.0: 690703,
         0.04: 273978,
         0.08: 29104,
         0.12: 5370,
         0.16: 787,
         0.2: 54,
         0.24: 4})

In [71]:
knn_res[:,1].shape

(1000000,)

In [75]:
knn_res_df = pd.DataFrame(knn_res[:,1])

In [82]:
knn_res_df

,0
0,0.00
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [86]:
idx = knn_res_df.nlargest(1000,[0]).index
idx

Int64Index([   523,  34962, 601215, 702443,  18472,  19651,  24124,  24134,
             26979,  30205,
            ...
             17966,  17968,  18075,  18097,  18185,  18250,  18458,  18712,
             18805,  18858],
           dtype='int64', length=1000)

In [89]:
submission_df = pd.DataFrame(test['customer'][idx])
submission_df.head()

,customer
523,91000523
34962,91034962
601215,91601215
702443,91702443
18472,91018472


In [90]:
submission_df.to_csv('knn_nlargest.csv')

In [38]:
knn.score(X,y)

0.9422801804776054

In [ ]:
knn_res[]

In [37]:
knn_res.shape

(1000000, 2)

In [121]:
Counter(knn_res)

Counter({0: 12594, 1: 987406})

In [116]:
Counter(knn_res)

Counter({0: 14275, 1: 985725})